### Capstone Project_Part2 (W5)

In [1]:
import numpy as np # library to handle data in a vectorized manner
from urllib.request import urlopen
content = urlopen("https://www.quora.com/")

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

##### Data Source - Imported with CSV file (Austrillia Population)

In [9]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_28b99893fe624894bdfa78a1e1b1f2c6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wUjjAJtVqTFfHkr16RvCwi6tNlvoygIcHzmHrflBYP04',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_28b99893fe624894bdfa78a1e1b1f2c6.get_object(Bucket='courseraibmcapstoneproject-donotdelete-pr-adydplayqlcwuc',Key='au.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data = pd.read_csv(body)
df_data.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Sydney,-33.861481,151.205475,Australia,AU,New South Wales,admin,4630000.0,3641422.0
1,South Melbourne,-37.833333,144.966667,Australia,AU,Victoria,NaN,4170000.0,93625.0
2,Brisbane,-27.471010,153.024292,Australia,AU,Queensland,admin,1860000.0,926353.0
3,Perth,-31.952240,115.861397,Australia,AU,Western Australia,admin,1532000.0,880216.0
4,Adelaide,-34.928661,138.598633,Australia,AU,South Australia,admin,1145000.0,836354.0


##### Remove unnecessary column - Data Wrangling

In [10]:
df_data.drop(columns = ["iso2", "capital","country", "population_proper"], inplace = True)
df_data.columns = ['City','Lat', 'Lng','State','Population']

df_australia = df_data[['City','Lat', 'Lng','State','Population']]
df_australia.head(5)

,City,Lat,Lng,State,Population
0,Sydney,-33.861481,151.205475,New South Wales,4630000.0
1,South Melbourne,-37.833333,144.966667,Victoria,4170000.0
2,Brisbane,-27.471010,153.024292,Queensland,1860000.0
3,Perth,-31.952240,115.861397,Western Australia,1532000.0
4,Adelaide,-34.928661,138.598633,South Australia,1145000.0


##### Choose State with most population in Australia

In [11]:
df_StateMax = df_australia.groupby('State')[['Population']].sum().sort_values(by = ['Population'], ascending = False).reset_index().head(1)
df_StateMax.head()

,State,Population
0,New South Wales,7020542.0


##### Visualization - Drawing City Map of New South Wales State

In [12]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium #Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

In [14]:
address = df_StateMax["State"][0]
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of {} are {} {}.'.format(
        address,latitude, longitude
    )
)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of New South Wales are -31.8759835 147.2869493.


##### Data Showing in New South Wales State

In [15]:
city_data = df_australia[df_australia['State'] == address].reset_index(drop=True)
city_data.head()

,City,Lat,Lng,State,Population
0,Sydney,-33.861481,151.205475,New South Wales,4630000.0
1,Newcastle,-32.927792,151.784485,New South Wales,1134616.0
2,Wollongong,-34.424000,150.893448,New South Wales,260914.0
3,Albury,-36.074823,146.924006,New South Wales,104258.0
4,Nowra,-34.884220,150.600357,New South Wales,94781.0


##### Drawing City Map of New South Wales State

In [16]:
# create map of New York using latitude and longitude values
map_NSW = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, State, City in zip(city_data['Lat'], city_data['Lng'], city_data['State'], city_data['City']):
    label = '{}, {}'.format(State, City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        
        fill_opacity=0.7,
        parse_html=False).add_to(map_NSW)  
    
map_NSW

##### Using Foursquare API

In [18]:
CLIENT_ID = 'BO2GWL4VTATKTGNCM3YNTSD0YGND3A3I3QICCIKJ0341RHGD' # your Foursquare ID
CLIENT_SECRET = '54GWQTS4DFQ1AGEYXAC4RZOCIFTYNYECI1SI2CHMEHT15BZG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BO2GWL4VTATKTGNCM3YNTSD0YGND3A3I3QICCIKJ0341RHGD
CLIENT_SECRET:54GWQTS4DFQ1AGEYXAC4RZOCIFTYNYECI1SI2CHMEHT15BZG


In [19]:
city_data.loc[0, 'City']

'Sydney'

Finding Venue in Sydney City using foursquare API

In [20]:
neighborhood_latitude = city_data.loc[0, 'Lat'] # neighborhood latitude value
neighborhood_longitude = city_data.loc[0, 'Lng'] # neighborhood longitude value

neighborhood_name = city_data.loc[0, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Sydney are -33.861481, 151.205475.


In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BO2GWL4VTATKTGNCM3YNTSD0YGND3A3I3QICCIKJ0341RHGD&client_secret=54GWQTS4DFQ1AGEYXAC4RZOCIFTYNYECI1SI2CHMEHT15BZG&v=20180605&ll=-33.861481,151.205475&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df88b8747e0d64ab92e2191'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sydney City Center',
  'headerFullLocation': 'Sydney City Center, Sydney',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': -33.856980995499995,
    'lng': 151.21088404680677},
   'sw': {'lat': -33.8659810045, 'lng': 151.20006595319325}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba864dcf964a5204ed939e3',
       'name': 'Sydney Harbour YHA',
       'location': {'address': '110 Cumberland St.',
        'lat': -33.860128396630415,
        'lng': 151.20684402123746,
        'labeledLatLngs': [{'label': 'display'

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sydney Harbour YHA,Hostel,-33.860128,151.206844
1,Shangri-La Hotel,Hotel,-33.861141,151.206460
2,The Langham Hotel Sydney,Hotel,-33.860517,151.203437
3,Harts Pub,Pub,-33.861870,151.206314
4,Blu Bar On 36,Hotel Bar,-33.861067,151.206361


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

98 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
 # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
df_venue=getNearbyVenues(names=city_data['City'],
                                   latitudes=city_data['Lat'],
                                   longitudes=city_data['Lng']
                                  )

Sydney
Newcastle
Wollongong
Albury
Nowra
Coffs Harbour
Wagga Wagga
Port Macquarie
Taree
Orange
West Tamworth
Tweed Heads
Queanbeyan
Dubbo
North Lismore
Armidale
Katoomba
Goulburn
East Maitland
Forster
Broken Hill
Griffith
Ballina
Richmond North
Singleton
Kempsey
Muswellbrook
Parkes
Lithgow
Batemans Bay
Kiama
South Grafton
Ulladulla
Inverell
Moree
Deniliquin
Young
Leeton
Gunnedah
Narrabri West
Cowra
Byron Bay
Tumut
Cooma
Pambula
Bathurst
Mudgee
Forbes
Scone
Central Coast
Bourke
Yamba
Wilcannia
Ivanhoe


In [29]:
df_venue.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sydney,-33.861481,151.205475,Sydney Harbour YHA,-33.860128,151.206844,Hostel
1,Sydney,-33.861481,151.205475,Shangri-La Hotel,-33.861141,151.206460,Hotel
2,Sydney,-33.861481,151.205475,The Langham Hotel Sydney,-33.860517,151.203437,Hotel
3,Sydney,-33.861481,151.205475,Harts Pub,-33.861870,151.206314,Pub
4,Sydney,-33.861481,151.205475,Blu Bar On 36,-33.861067,151.206361,Hotel Bar


In [30]:
print(df_venue.shape)

df_merged = pd.merge(df_venue, city_data, left_on=['City'],
              right_on=['City'],
              how='inner')

df_mergedFinal = df_merged[["City","Population","City Latitude","City Longitude","Venue","Venue Latitude","Venue Longitude","Venue Category"]]
df_mergedFinal.head(5)

(647, 7)


,City,Population,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sydney,4630000.0,-33.861481,151.205475,Sydney Harbour YHA,-33.860128,151.206844,Hostel
1,Sydney,4630000.0,-33.861481,151.205475,Shangri-La Hotel,-33.861141,151.206460,Hotel
2,Sydney,4630000.0,-33.861481,151.205475,The Langham Hotel Sydney,-33.860517,151.203437,Hotel
3,Sydney,4630000.0,-33.861481,151.205475,Harts Pub,-33.861870,151.206314,Pub
4,Sydney,4630000.0,-33.861481,151.205475,Blu Bar On 36,-33.861067,151.206361,Hotel Bar


In [31]:
print('The dataframe has {} cities and {} Venue Category.'.format(
        len(df_mergedFinal['City'].unique()),len(df_mergedFinal['Venue Category'].unique())
    )
)

The dataframe has 49 cities and 124 Venue Category.


##### City Selection - choose city with most population

In [32]:
row_num = df_mergedFinal['Population'].argmax()
city_name = df_mergedFinal['City'].iloc[row_num]
city_name

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  if __name__ == '__main__':


'Sydney'

In [33]:
lat_Sydney = df_mergedFinal['City Latitude'].iloc[row_num]
long_Sydney = df_mergedFinal['City Longitude'].iloc[row_num]
print(lat_Sydney, long_Sydney)

-33.861481 151.205475


Using FourSquare API to get Venues in Sydney of New South Wales State

In [34]:
dfSydney_Veneu = df_venue[df_venue['City'].str.contains(city_name)]    
dfSydney_Veneu.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sydney,-33.861481,151.205475,Sydney Harbour YHA,-33.860128,151.206844,Hostel
1,Sydney,-33.861481,151.205475,Shangri-La Hotel,-33.861141,151.206460,Hotel
2,Sydney,-33.861481,151.205475,The Langham Hotel Sydney,-33.860517,151.203437,Hotel
3,Sydney,-33.861481,151.205475,Harts Pub,-33.861870,151.206314,Pub
4,Sydney,-33.861481,151.205475,Blu Bar On 36,-33.861067,151.206361,Hotel Bar


In [35]:
print('The dataframe has {} cities and {} venues'.format(
        len(dfSydney_Veneu['City'].unique()),len(dfSydney_Veneu['Venue Category'].unique())
    )
)

The dataframe has 1 cities and 50 venues


In [36]:
dfSydney_Veneu['Venue Category'].unique()

array(['Hostel', 'Hotel', 'Pub', 'Hotel Bar', 'Café', 'Park',
       'Planetarium', 'Italian Restaurant', 'Restaurant', 'Coffee Shop',
       'Sandwich Place', 'Scenic Lookout', 'Pizza Place', 'Cocktail Bar',
       'Bakery', 'Seafood Restaurant', 'Chinese Restaurant', 'Brewery',
       'Art Museum', 'Speakeasy', 'Flea Market', 'Sushi Restaurant',
       'History Museum', 'Japanese Restaurant', 'Gym / Fitness Center',
       'Ice Cream Shop', 'Thai Restaurant', 'Food Court',
       'Tapas Restaurant', 'Australian Restaurant', 'Burger Joint',
       'Tour Provider', 'Wine Bar', 'Performing Arts Venue', 'Theater',
       'Gym', 'Steakhouse', 'Bistro', 'Creperie', 'Donut Shop',
       'Dessert Shop', 'German Restaurant', 'Bar', 'Lounge',
       'Middle Eastern Restaurant', 'Museum', 'Tunnel', 'Bed & Breakfast',
       'Train Station', 'Vietnamese Restaurant'], dtype=object)

Analyze Each City with Venue Category

In [37]:
k = dfSydney_Veneu.copy(deep = True)
new_weightage_dict= {
     'Theater':6,
 'Lounge':1,
 'Creperie':1,
 'Museum':1,
 'Art Museum':1,
 'History Museum':1,
 'Planetarium':1,
 'Performing Arts Venue':1,
 'Tunnel':1,
 'Scenic Lookout':1,
 'Speakeasy':1,
 'Italian Restaurant':2,
 'Seafood Restaurant':2,
 'Sushi Restaurant':2,
 'Restaurant':2,
 'Chinese Restaurant':2,
 'Tapas Restaurant':2,
 'Middle Eastern Restaurant':2,
 'Japanese Restaurant':2,
 'Thai Restaurant':2,
 'Australian Restaurant':2,
 'German Restaurant':2,
 'Vietnamese Restaurant':2,
 'Hotel Bar':6,
 'Hotel':3,
 'Cocktail Bar':3,
 'Brewery':3,
'Hostel':3,
 'Wine Bar':3,
 'Coffee Shop':4,
 'Donut Shop':4,
 'Burger Joint':4,
 'Pizza Place':4,
 'Café':4,
 'Bed & Breakfast':4,
 'Bakery':4,
 'Sandwich Place':4,
 'Ice Cream Shop':4,
 'Dessert Shop':4,
 'Steakhouse':4,
 'Bar':4,
 'Park':5,
 'Gym':5,
 'Gym / Fitness Center':5,
 'Pub':5,
 'Bistro':5,
 'Flea Market':5,
 'Train Station':5,
 'Tour Provider':6,
 'Food Court':6
}

##### Plotting City (Sydney)

In [38]:
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_Sydney, long_Sydney], zoom_start=15) # generate map centred around Sydney city


# add Jersey City as a red circle mark
folium.features.CircleMarker(
    [lat_Sydney, long_Sydney],
    radius=10,
    popup='Sydney city',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [41]:
dfSydney_Veneu.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sydney,-33.861481,151.205475,Sydney Harbour YHA,-33.860128,151.206844,Hostel
1,Sydney,-33.861481,151.205475,Shangri-La Hotel,-33.861141,151.206460,Hotel
2,Sydney,-33.861481,151.205475,The Langham Hotel Sydney,-33.860517,151.203437,Hotel
3,Sydney,-33.861481,151.205475,Harts Pub,-33.861870,151.206314,Pub
4,Sydney,-33.861481,151.205475,Blu Bar On 36,-33.861067,151.206361,Hotel Bar


In [40]:
# add all the venue of the Sydney city to the map as blue circle markers
for lat, lng, label in zip(dfSydney_Veneu['Venue Latitude'], dfSydney_Veneu['Venue Longitude'], dfSydney_Veneu['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

In [75]:
allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Sydney,-33.861481,151.205475,Sydney Harbour YHA,-33.860128,151.206844,Hostel,3
1,Sydney,-33.861481,151.205475,Shangri-La Hotel,-33.861141,151.206460,Hotel,3
2,Sydney,-33.861481,151.205475,The Langham Hotel Sydney,-33.860517,151.203437,Hotel,3
3,Sydney,-33.861481,151.205475,Harts Pub,-33.861870,151.206314,Pub,5
4,Sydney,-33.861481,151.205475,Blu Bar On 36,-33.861067,151.206361,Hotel Bar,6


In [83]:
newframe = k[['Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["City Latitude", "City Longitude"], inplace = True)
newframe.head(5)

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
Art Museum,-33.859859,151.209018,1
Australian Restaurant,-33.860805,151.206250,2
Bakery,-33.861649,151.207965,4
Bar,-33.865069,151.203811,4
Bed & Breakfast,-33.865210,151.208464,4


##### Apply K mean cluster to group venue category

In [84]:
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
city_df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(city_df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,Art Museum,1,-33.859859,151.209018,0
1,Australian Restaurant,2,-33.860805,151.206250,0
2,Bakery,4,-33.861649,151.207965,2
3,Bar,4,-33.865069,151.203811,1
4,Bed & Breakfast,4,-33.865210,151.208464,1


In [91]:
#new group by clusters and add weights of each cluster 
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,1.736842,-33.860244,151.207538
1,3.315789,-33.863634,151.206199
2,5.000000,-33.860205,151.208122


In [92]:
# Choose with maximum weight
lat1 =-33.860205
long1 =151.208122

In [96]:
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Sydney city

# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=100,
    popup='Hotel can be opened within this circle',
    fill=True,
    color='red',
    fill_color='#3186cc',
    fill_opacity=0.2
    ).add_to(final_map)
final_map

### Summary

Best City in New South Wales of Austrilia for opening Hotel - Sydney 

Best Locality in Sydney - on Harrington Street (near red circle)